# Import Required Libraries
Import the necessary libraries, including roadrunner, matplotlib, numpy, multiprocessing, and ipywidgets.

In [ ]:
import roadrunner
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mp
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
# Define the RoadRunner instance
rr = roadrunner.RoadRunner("SimpleFALiver.xml")

# Define Helper Functions
Define the helper functions, including get_steady_state and plot_simulation.

In [ ]:
# Define helper function to get steady state values
def get_steady_state(fax: float):
    rr.reset()
    rr.M_FAx = fax
    rr.steadyStateSelections = ["M_LD", "M_TAG", "M_FA"]
    return rr.getSteadyStateValues()

# Define helper function to plot simulation results
def plot_simulation(max_time: float, initial_fax: float):
    rr.reset()
    rr.M_FAx = initial_fax
    rr.simulate(0, max_time, 200, selections=["time", "M_LD", "M_TAG", "M_FA"])
    fig, ax = plt.subplots()
    rr.plot(show=False)
    ax.legend()
    ax.set_xlabel("Time")
    ax.set_ylabel("Concentration")
    fig.tight_layout()
    plt.show()

# Set Up Interactive Widgets
Set up interactive widgets using ipywidgets for selecting the maximum time for the timecourse simulation and the initial concentration of FAx.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

# Set up interactive widgets for selecting max time and initial FAx concentration
max_time_slider = widgets.FloatSlider(
    value=5.0,
    min=1.0,
    max=10.0,
    step=0.1,
    description='Max Time:',
    continuous_update=False
)

initial_fax_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.1,
    description='Initial FAx:',
    continuous_update=False
)

# Use interact to link the sliders with the plot_simulation function
interact(plot_simulation, max_time=max_time_slider, initial_fax=initial_fax_slider)

# Run Timecourse Simulation
Run the timecourse simulation based on the selected maximum time and plot the results.

In [ ]:
# Run Timecourse Simulation
# Run the timecourse simulation based on the selected maximum time and plot the results.

# Set up interactive widgets for selecting max time and initial FAx concentration
max_time_slider = widgets.FloatSlider(
    value=5.0,
    min=1.0,
    max=10.0,
    step=0.1,
    description='Max Time:',
    continuous_update=False
)

initial_fax_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.1,
    description='Initial FAx:',
    continuous_update=False
)

# Use interact to link the sliders with the plot_simulation function
interact(plot_simulation, max_time=max_time_slider, initial_fax=initial_fax_slider)

# Run Steady State Simulation
Run the steady state simulation based on the selected initial concentration of FAx and plot the results.

In [ ]:
# Run Steady State Simulation
# Run the steady state simulation based on the selected initial concentration of FAx and plot the results.

# Define helper function to run steady state simulation and plot results
def run_steady_state_simulation(range_fax: float):
    result_dict = {
        "M_FAx": list(),
        "M_LD": list(),
        "M_TAG": list(),
        "M_FA": list()
    }
    fax_range = np.arange(range_fax[0], range_fax[1], .01)
    
    with mp.Pool(mp.cpu_count()) as pool:
        results = pool.map(
            get_steady_state,
            fax_range
        )
    
    for fax, res in zip(fax_range, results):
        result_dict["M_FAx"].append(fax)
        result_dict["M_LD"].append(res[0])
        result_dict["M_TAG"].append(res[1])
        result_dict["M_FA"].append(res[2])
    
    fig, ax = plt.subplots()
    ax.plot(result_dict["M_FAx"], result_dict["M_LD"], label="LD")
    ax.plot(result_dict["M_FAx"], result_dict["M_TAG"], label="TAG")
    ax.plot(result_dict["M_FAx"], result_dict["M_FA"], label="FA")
    ax.legend()
    ax.set_xlabel("FAx")
    ax.set_ylabel("Concentration")
    fig.tight_layout()
    plt.show()

# Set up interactive widget for selecting FAx concentration range

range_fax_slider = widgets.FloatRangeSlider(
    value=[0.0, 5.0],
    min=0.0,
    max=5.0,
    step=0.1,
    description='Range FAx:',
    continuous_update=False
)

# Use interact to link the slider with the initial_fax=initial_fax_slider_steady_state)
interact(run_steady_state_simulation, range_fax=range_fax_slider)

# Display Plots
Display the plots for both the timecourse and steady state simulations.

In [ ]:
# Display Plots
# Display the plots for both the timecourse and steady state simulations.

# Plot Timecourse Simulation
interact(plot_simulation, max_time=max_time_slider, initial_fax=initial_fax_slider)

# Plot Steady State Simulation
interact(run_steady_state_simulation, range_fax=range_fax_slider)